###Importação de bibliotecas

In [ ]:
# Importação básica
import os

In [2]:
# Importações de visualização
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Importação para visualização de imagem
from PIL import Image, ImageOps
from tqdm import tqdm

# Importação para carregar o Dataset
import gdown
import pickle
import zipfile
from google.colab import files

In [3]:
# Importação de preparação
import random

# Importação da Grad-Cam
import cv2

In [4]:
# Importações de arquitetura
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import DenseNet121

# Importação de validação
from sklearn.model_selection import StratifiedKFold

###Carregamento de dataset e pesos

In [5]:
# Função de criação de dataframe
def create_df(rows, path):
    for label in ["NORMAL", "PNEUMONIA"]:
        folder = os.path.join(path, label)

        if not os.path.exists(folder):
            continue

        for filename in os.listdir(folder):
            if filename.lower().endswith((".jpeg", ".jpg", ".png")):
                rows.append({
                    "filepath": os.path.join(folder, filename),
                    "filename": filename,
                    "label": label,
                    "label_bin": 0 if label == "NORMAL" else 1,
                    })

    df = pd.DataFrame(rows)
    return df

In [6]:
# Este notebook é exclusivo para modelagem, logo o dataset será recarregado
# Carregamento das imagens de teste
file_test_id = "1IBC3mk83DnHkZ4Xn3Kq9H18kM3_WITx2"

gdown.download(
    f"https://drive.google.com/uc?id={file_test_id}",
    "/content/chest_xray.zip",
    quiet=False,
    )

with zipfile.ZipFile("/content/chest_xray.zip", "r") as zip_ref:
    zip_ref.extractall("/content/data_test")

print("Dataset carregado!")

Downloading...
From (original): https://drive.google.com/uc?id=1IBC3mk83DnHkZ4Xn3Kq9H18kM3_WITx2
From (redirected): https://drive.google.com/uc?id=1IBC3mk83DnHkZ4Xn3Kq9H18kM3_WITx2&confirm=t&uuid=3cb22d5d-7a28-4451-ab37-dfcc6fe159a3
To: /content/chest_xray.zip
100%|██████████| 1.26G/1.26G [00:20<00:00, 62.9MB/s]


Dataset carregado!


In [7]:
# Criação do dataframe de teste
rows_test = []
test_path = "/content/data_test/chest_xray/test"

df_test = create_df(rows_test, test_path)
print(f"Treino: {len(df_test)} imagens")
print(df_test["label"].value_counts())

Treino: 624 imagens
label
PNEUMONIA    390
NORMAL       234
Name: count, dtype: int64


In [8]:
# Carregamento das imagens de treino tratado
file_train_id = "1AGEiWDB7BaZA0qrTSsZOkMqM7fd9ru4e"

gdown.download(
    f"https://drive.google.com/uc?id={file_train_id}",
    "/content/chest_xray_tratado.zip",
    quiet=False,
    )

with zipfile.ZipFile("/content/chest_xray_tratado.zip", "r") as zip_ref:
    zip_ref.extractall("/content/data_train_tratado")

print("Dataset carregado!")

Downloading...
From (original): https://drive.google.com/uc?id=1AGEiWDB7BaZA0qrTSsZOkMqM7fd9ru4e
From (redirected): https://drive.google.com/uc?id=1AGEiWDB7BaZA0qrTSsZOkMqM7fd9ru4e&confirm=t&uuid=588430f9-5627-404a-bc46-c092acf7a4cd
To: /content/chest_xray_tratado.zip
100%|██████████| 40.3M/40.3M [00:00<00:00, 61.0MB/s]


Dataset carregado!


In [9]:
# Criação do dataframe de treino
rows_train = []
train_path = "/content/data_train_tratado/chest_xray_tratado/train"

df_train = create_df(rows_train, train_path)
print(f"Treino: {len(df_train)} imagens")
print(df_train["label"].value_counts())

Treino: 5206 imagens
label
PNEUMONIA    3858
NORMAL       1348
Name: count, dtype: int64


In [10]:
# Carregamento dos pesos
weight_id = "1S3Lh7Ha2rqbCgNPkxinIqdcQ_L_qpS_U"

gdown.download(
    f"https://drive.google.com/uc?id={weight_id}",
    "/content/chest_xray_weight.pkl",
    quiet=False,
    )

with open('/content/chest_xray_weight.pkl', 'rb') as f:
    class_weights = pickle.load(f)

Downloading...
From: https://drive.google.com/uc?id=1S3Lh7Ha2rqbCgNPkxinIqdcQ_L_qpS_U
To: /content/chest_xray_weight.pkl
100%|██████████| 38.0/38.0 [00:00<00:00, 175kB/s]


###Definição de funções auxiliares

In [11]:
# Função de normalização dos pixels + conversão para grayscale
def carregar_imagem(filepath):
    if hasattr(filepath, 'numpy'):
        filepath = filepath.numpy().decode('utf-8')

    img = Image.open(filepath).convert('L')
    img = ImageOps.pad(img, (224, 224), color=0)
    arr = np.array(img, dtype=np.float32)
    arr = arr / 255.0
    arr = (arr - 0.485) / 0.229
    arr = np.stack([arr, arr, arr], axis=-1)
    return arr

In [12]:
# Função de augmentação (auxílio ao OverSampling)
def augmentar_imagem(img):
    if random.random() > 0.5:
        img = ImageOps.mirror(img)

    angulo = random.uniform(-10, 10)
    img = img.rotate(angulo, fillcolor=0)

    margem = random.uniform(0.05, 0.10)
    w, h = img.size
    img = img.crop((
        int(w * margem),
        int(h * margem),
        int(w * (1 - margem)),
        int(h * (1 - margem)),
    ))
    img = img.resize((224, 224), Image.LANCZOS)

    return img

In [13]:
# Uso apenas em casos de treino
# As duas funções "juntas"
def carregar_com_augmentation(filepath):
    if hasattr(filepath, 'numpy'):
        filepath = filepath.numpy().decode('utf-8')

    img = Image.open(filepath).convert('L')
    img = augmentar_imagem(img)
    arr = np.array(img, dtype=np.float32)
    arr = arr / 255.0
    arr = (arr - 0.485) / 0.229
    arr = np.stack([arr, arr, arr], axis=-1)
    return arr

In [14]:
# Gerador para carregar uma parte do dataset por vez e usar menos memória RAM
def criar_gerador(df, batch_size=32, augmentar=False):
    dataset = tf.data.Dataset.from_tensor_slices(
        (df['filepath'].values, df['label_bin'].values),
        )

    def carregar_batch(filepath, label):
        img = tf.py_function(
            func = carregar_com_augmentation if augmentar else carregar_imagem,
            inp  = [filepath],
            Tout = tf.float32,
            )
        img.set_shape((224, 224, 3))
        return img, label

    dataset = dataset.map(carregar_batch, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

###Arquitetura do modelo

Com o intuito de ter um bom desempenho, foi escolhida a rede convolucional DenseNet121 para compor a modelagem do projeto. Essa decisão levou em conta a popularidade da rede CNN em projetos do eixo médico, tendo bom desempenhos em métricas de Recall e AUC-ROC com aplicação na área de visão computacional.

In [15]:
# Função da arquitetura (personalizável)
def construir_modelo():
    base_model = DenseNet121(
        # Os pesos são públicos, respeitando as regras de uso de transfer learning
        weights = 'imagenet',
        include_top = False,
        input_shape = (224, 224, 3),
        )

    base_model.trainable = False

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=x)

    return model, base_model

model, base_model = construir_modelo()

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [16]:
# Função para compilar
def compilar_modelo(model, learning_rate):
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss      = 'binary_crossentropy',
        metrics   = [
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.Precision(name='precision'),
            ])
    return model

###Cross-Validation

Na primeira tentativa de rodar o código abaixo, ele carregava todas as imagens de uma vez, o que consumiu toda a memória RAM disponível. A solução achada foi criar um meio de carregamento em lotes e temporário.

In [17]:
tf.random.set_seed(42)
np.random.seed(42)

historico_auc = []
historico_recall = []
modelos_por_fold = []

# O K = 5 foi escolhido como valor padrão
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(df_train, df_train['label_bin'])):
    print(f"FOLD {fold + 1} / 5")

    # Um subconjunto de validação é criado a partir do conjunto de treino
    df_fold_train = df_train.iloc[train_idx].reset_index(drop=True)
    df_fold_val = df_train.iloc[val_idx].reset_index(drop=True)

    print(f"Treino: {len(df_fold_train)} imagens")
    print(f"Val: {len(df_fold_val)} imagens")

    print("Carregando imagens...")

    # Augmention aplicada no conjunto de treino
    train_gen = criar_gerador(df_fold_train, batch_size=32, augmentar=True)
    val_gen = criar_gerador(df_fold_val, batch_size=32, augmentar=False)

    # Passa pelas funções de criar modelo + compilação
    model, base_model = construir_modelo()
    model = compilar_modelo(model, learning_rate=1e-4)

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor = 'val_auc',
            patience = 3,
            mode = 'max',
            restore_best_weights = True,
            ),

        tf.keras.callbacks.ReduceLROnPlateau(
            monitor = 'val_auc',
            factor = 0.5,
            patience = 2,
            mode = 'max',
            verbose = 1,
            )]

    # Feature extraction
    print("Fase 1 — Feature extraction")

    history_fase1 = model.fit(
        train_gen,
        validation_data = val_gen,
        epochs = 15,
        class_weight = class_weights,
        callbacks = callbacks,
        verbose = 1,
        )

    # Fine tuning
    print("Fase 2 — Fine tuning")

    # Descongela as últimas 45 camadas da DenseNet
    for layer in base_model.layers[-45:]:
        layer.trainable = True

    model = compilar_modelo(model, learning_rate=1e-5)

    history_fase2 = model.fit(
        train_gen,
        validation_data = val_gen,
        epochs = 20,
        class_weight = class_weights,
        callbacks = callbacks,
        verbose = 1,
        )

    # Métricas
    resultados = model.evaluate(val_gen, verbose=0)
    auc_fold = resultados[1]
    recall_fold = resultados[2]

    historico_auc.append(auc_fold)
    historico_recall.append(recall_fold)
    modelos_por_fold.append(model)

    print(f"Fold {fold+1} concluído:")
    print(f"  AUC-ROC: {auc_fold:.4f}")
    print(f"  Recall:  {recall_fold:.4f}")

FOLD 1 / 5
Treino: 4164 imagens
Val: 1042 imagens
Carregando imagens...
Fase 1 — Feature extraction
Epoch 1/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 91s 472ms/step - auc: 0.5646 - loss: 1.3088 - precision: 0.5377 - recall: 0.4361 - val_auc: 0.5000 - val_loss: 2.0026 - val_precision: 0.7409 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 2/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 24s 184ms/step - auc: 0.4042 - loss: 2.4240 - precision: 0.4178 - recall: 0.4936 - val_auc: 0.5764 - val_loss: 1.5370 - val_precision: 0.7409 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 3/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 24s 181ms/step - auc: 0.4416 - loss: 1.9145 - precision: 0.4253 - recall: 0.5500 - val_auc: 0.7748 - val_loss: 1.3457 - val_precision: 0.7409 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 4/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 24s 180ms/step - auc: 0.4957 - loss: 1.5160 - precision: 0.4488 - recall: 0.5792 - val_auc: 0.8223 - val_loss: 1.2630 - val_precision: 0.7409 - val_recall: 1.0000 - le

In [18]:
print(f"\n{'='*45}")
print(f"   RESULTADOS FINAIS — K-FOLD")
print(f"{'='*45}\n")

for i, (auc, recall) in enumerate(zip(historico_auc, historico_recall)):
    print(f"Fold {i+1}: AUC={auc:.4f}  Recall={recall:.4f}")

print(f"\n📊 Médias:")
print(f"   AUC-ROC: {np.mean(historico_auc):.4f} ± {np.std(historico_auc):.4f}")
print(f"   Recall:  {np.mean(historico_recall):.4f} ± {np.std(historico_recall):.4f}")

# Identifica o melhor fold pelo AUC
melhor_fold = np.argmax(historico_auc)
print(f"\n🏆 Melhor fold: Fold {melhor_fold+1}")
print(f"   AUC-ROC: {historico_auc[melhor_fold]:.4f}")
print(f"   Recall:  {historico_recall[melhor_fold]:.4f}")

# Seleciona o melhor modelo
melhor_modelo = modelos_por_fold[melhor_fold]


   RESULTADOS FINAIS — K-FOLD

Fold 1: AUC=0.9692  Recall=0.9987
Fold 2: AUC=0.9709  Recall=1.0000
Fold 3: AUC=0.9695  Recall=1.0000
Fold 4: AUC=0.9702  Recall=1.0000
Fold 5: AUC=0.9741  Recall=1.0000

📊 Médias:
   AUC-ROC: 0.9708 ± 0.0018
   Recall:  0.9997 ± 0.0005

🏆 Melhor fold: Fold 5
   AUC-ROC: 0.9741
   Recall:  1.0000


###Treinamento final

In [19]:
print("   TREINAMENTO FINAL — DATASET COMPLETO")

train_gen_final = criar_gerador(df_train, batch_size=32, augmentar=True)

model_final, base_model_final = construir_modelo()
model_final = compilar_modelo(model_final, learning_rate=1e-4)

# Callbacks ajustados — sem validação, usa apenas loss de treino
callbacks_final = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor = 'loss',
        factor = 0.5,
        patience = 3,
        mode = 'min',
        verbose = 1,
        )]

# Fase 1 — Feature extraction
print("Fase 1 — Feature extraction")
model_final.fit(
    train_gen_final,
    epochs = 15,
    class_weight = class_weights,
    callbacks = callbacks_final,
    verbose = 1,
    )

# Fase 2 — Fine tuning
print("\nFase 2 — Fine tuning")
for layer in base_model_final.layers[-45:]:
    layer.trainable = True

model_final = compilar_modelo(model_final, learning_rate=1e-5)

model_final.fit(
    train_gen_final,
    epochs = 20,
    class_weight = class_weights,
    callbacks = callbacks_final,
    verbose = 1,
    )

print("\n✅ Modelo final treinado!")

   TREINAMENTO FINAL — DATASET COMPLETO
Fase 1 — Feature extraction
Epoch 1/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 64s 243ms/step - auc: 0.6256 - loss: 1.0960 - precision: 0.6354 - recall: 0.4823 - learning_rate: 1.0000e-04
Epoch 2/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 25s 152ms/step - auc: 0.4661 - loss: 2.8568 - precision: 0.4507 - recall: 0.4998 - learning_rate: 1.0000e-04
Epoch 3/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 24s 150ms/step - auc: 0.5035 - loss: 1.9252 - precision: 0.4539 - recall: 0.5331 - learning_rate: 1.0000e-04
Epoch 4/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 25s 151ms/step - auc: 0.5309 - loss: 1.6108 - precision: 0.4663 - recall: 0.5575 - learning_rate: 1.0000e-04
Epoch 5/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 25s 151ms/step - auc: 0.5770 - loss: 1.2165 - precision: 0.4875 - recall: 0.6079 - learning_rate: 1.0000e-04
Epoch 6/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 24s 149ms/step - auc: 0.5879 - loss: 1.1859 - precision: 0.5006 - recall: 0.6179 - learning_rate: 1.0000e-04
Epoch 7/15
163/163 ━━━━━━━━━━━━━━━━━━━

In [25]:
# Salva o modelo treinado
model_final.save('/content/modelo_pneumonia_final.keras')
print("✅ Modelo salvo em: modelo_pneumonia_final.keras")

# Faz download
from google.colab import files
files.download('/content/modelo_pneumonia_final.keras')

✅ Modelo salvo em: modelo_pneumonia_final.keras


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###Predições

In [27]:
# Análise do que foi treinado com base no teste
predicoes = []

for filepath in df_test['filepath']:
    img = carregar_imagem(filepath)
    img = np.expand_dims(img, axis=0)
    prob = model_final.predict(img, verbose=0)[0][0]
    predicoes.append(prob)

df_test['prob_pneumonia'] = predicoes

print("\n✅ Predições concluídas!")
print(df_test.head(20))

df_test.to_csv('/content/predicoes_teste.csv', index=False)
files.download('/content/predicoes_teste.csv')


✅ Predições concluídas!
                                             filepath  \
0   /content/data_test/chest_xray/test/NORMAL/NORM...   
1   /content/data_test/chest_xray/test/NORMAL/NORM...   
2   /content/data_test/chest_xray/test/NORMAL/NORM...   
3   /content/data_test/chest_xray/test/NORMAL/NORM...   
4   /content/data_test/chest_xray/test/NORMAL/NORM...   
5   /content/data_test/chest_xray/test/NORMAL/NORM...   
6   /content/data_test/chest_xray/test/NORMAL/NORM...   
7   /content/data_test/chest_xray/test/NORMAL/NORM...   
8   /content/data_test/chest_xray/test/NORMAL/NORM...   
9   /content/data_test/chest_xray/test/NORMAL/NORM...   
10  /content/data_test/chest_xray/test/NORMAL/NORM...   
11  /content/data_test/chest_xray/test/NORMAL/NORM...   
12  /content/data_test/chest_xray/test/NORMAL/NORM...   
13  /content/data_test/chest_xray/test/NORMAL/NORM...   
14  /content/data_test/chest_xray/test/NORMAL/NORM...   
15  /content/data_test/chest_xray/test/NORMAL/NORM...   
16  /c

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
# Verifica se TODAS as imagens são realmente NORMAL
print("Contagem de labels:")
print(df_test['label'].value_counts())

# Verifica a distribuição das probabilidades
print(f"\nEstatísticas de prob_pneumonia:")
print(f"Mínima:  {df_test['prob_pneumonia'].min():.4f}")
print(f"Máxima:  {df_test['prob_pneumonia'].max():.4f}")
print(f"Média:   {df_test['prob_pneumonia'].mean():.4f}")
print(f"Mediana: {df_test['prob_pneumonia'].median():.4f}")

# Quantas o modelo classificaria como PNEUMONIA usando limiar 0.5?
n_pneumonia = (df_test['prob_pneumonia'] >= 0.5).sum()
print(f"\nClassificadas como PNEUMONIA (prob >= 0.5): {n_pneumonia}/{len(df_test)}")

Contagem de labels:
label
PNEUMONIA    390
NORMAL       234
Name: count, dtype: int64

Estatísticas de prob_pneumonia:
Mínima:  0.6867
Máxima:  0.9977
Média:   0.9537
Mediana: 0.9625

Classificadas como PNEUMONIA (prob >= 0.5): 624/624
